In [ ]:
import pandas as pd
import numpy as np
'''
***Libraries/Modules and their Uses***
Pandas for handling datasetsand Numpy for managing Nan values
'''

In [ ]:
'''Creating seperate columns for Month and Year for merging datasets and categorising seasons as Rabi Kharif Winter Summer'''

df_aqi=pd.read_csv('aqi.csv',encoding='latin1')
df_aqi.head()

df_aqi['year']=pd.to_datetime(df_aqi['date'],format="%Y-%m-%d").dt.year
df_aqi['month']=pd.to_datetime(df_aqi['date'],format="%Y-%m-%d").dt.month
df_aqi.drop(columns='date',inplace=True)
df_aqi.head()
print(df_aqi.shape)

C:\Users\anura\AppData\Local\Temp\ipykernel_17904\2724316619.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_aqi=pd.read_csv('aqi.csv',encoding='latin1')


(435742, 14)


In [ ]:

'''Mapping the seasons and grouping AQI Values by Season, after this cell, the aqi.csv is modified to final_aqi.csv'''
def get_season(month):
    if month in [6, 7, 8, 9]:
        return 'Kharif'
    elif month in [10, 11]:
        return 'Rabi'
    elif month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Summer'
    else:
        return 'Unknown'


df_aqi['season'] = df_aqi['month'].apply(get_season)

pollutants = ['so2', 'no2', 'rspm', 'spm', 'pm2_5']

df_aqi[pollutants] = df_aqi[pollutants].fillna(0)

seasonal_df = df_aqi.groupby(['year', 'state', 'season'])[pollutants].sum().reset_index()

yearly_df = df_aqi.groupby(['year', 'state'])[pollutants].sum().reset_index()
yearly_df['season'] = 'Whole Year'

final_aqi_df = pd.concat([seasonal_df, yearly_df], ignore_index=True)

final_aqi_df = final_aqi_df.sort_values(['year', 'state', 'season'])
final_aqi_df['year'] = final_aqi_df['year'].astype(int)

final_aqi_df.to_csv("final_aqi.csv", index=False)

final_aqi_df.head()

,year,state,season,so2,no2,rspm,spm,pm2_5
0,1987,Bihar,Kharif,107.3,276.0,0.0,2095.0,0.0
1,1987,Bihar,Rabi,23.6,21.0,0.0,227.0,0.0
2,1987,Bihar,Summer,228.5,171.2,0.0,2745.0,0.0
2546,1987,Bihar,Whole Year,440.2,610.5,0.0,5942.0,0.0
3,1987,Bihar,Winter,80.8,142.3,0.0,875.0,0.0


In [ ]:
'''Renaming AQI columns to match crops data'''

final_aqi_df.rename(columns={'state':'State_Name','year':'Crop_Year'},inplace=True)
final_aqi_df.to_csv("final_aqi.csv", index=False)
final_aqi_df.head()

,Crop_Year,State_Name,season,so2,no2,rspm,spm,pm2_5
0,1987,Bihar,Kharif,107.3,276.0,0.0,2095.0,0.0
1,1987,Bihar,Rabi,23.6,21.0,0.0,227.0,0.0
2,1987,Bihar,Summer,228.5,171.2,0.0,2745.0,0.0
2546,1987,Bihar,Whole Year,440.2,610.5,0.0,5942.0,0.0
3,1987,Bihar,Winter,80.8,142.3,0.0,875.0,0.0


In [74]:
df_rainfall=pd.read_csv("rainfall.csv",encoding='latin1')   

In [ ]:
'''Mapping rainfall according to seasons and further changing rainfall.csv to redefined_rainfall.csv'''

df_rainfall['Winter']=df_rainfall['DEC'] + df_rainfall['JAN'] + df_rainfall['FEB']
df_rainfall['Summer']=df_rainfall['MAR'] + df_rainfall['APR'] + df_rainfall['MAY']
df_rainfall['Kharif']=df_rainfall['JJAS']
df_rainfall['Rabi']=df_rainfall['OCT'] + df_rainfall['NOV']
df_rainfall.drop(columns=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','MAM','NOV','DEC','JF','OND','JJAS'],inplace=True)
df_rainfall.rename(columns={'SUBDIVISION':'State_Name'},inplace=True)
df_rainfall.to_csv('redefined_rainfall.csv',index=True)
df_rainfall.rename(columns={'YEAR':'Crop_Year'},inplace=True)
df_rainfall.rename(columns={'ANNUAL':'Whole Year'},inplace=True)


In [ ]:
'''Preparing Rainfall dataset for the first merge, the state_names were inconsistent in both the datasets causing error, so had to 
change rainfall dataset to match crops, this was done by identifying the discrepencies after printing the unique columns of both the datasets,
then correcting them'''
rainfall_mapping = {
    "Assam & Meghalaya": ["Assam", "Meghalaya"],
    "Orissa": "Odisha",
    "Jammu and Kashmir ": "Jammu and Kashmir",  
    "Gujarat Region": "Gujarat",
    "West Uttar Pradesh": "Uttar Pradesh",
    "East Uttar Pradesh": "Uttar Pradesh",
    "West Rajasthan": "Rajasthan",
    "East Rajasthan": "Rajasthan",
    "West Madhya Pradesh": "Madhya Pradesh",
    "East Madhya Pradesh": "Madhya Pradesh",
    "Madhya Maharashtra": "Maharashtra",
    "Matathwada": "Maharashtra",
    "Vidarbha": "Maharashtra",
    "Rayalseema": "Andhra Pradesh",
    "Gangetic West Bengal": "West Bengal",
    "Coastal Andhra Pradesh": "Andhra Pradesh",
    "Saurashtra": "Gujarat",
    "Kutch": "Gujarat",
    "Coastal Karnataka": "Karnataka",
    "North Interior Karnataka": "Karnataka",
    "South Interior Karnataka": "Karnataka"
}

def fix_rainfall_states(df_rainfall):
    expanded_rows = []
    
    for _, row in df_rainfall.iterrows():
        state_name = row["State_Name"]
        
        if state_name in rainfall_mapping:
            new_states = rainfall_mapping[state_name]
            if isinstance(new_states, list):
                for state in new_states:
                    new_row = row.copy()
                    new_row["State_Name"] = state
                    expanded_rows.append(new_row)
            else:
                row["State_Name"] = new_states
                expanded_rows.append(row)
        else:
            expanded_rows.append(row)

    return pd.DataFrame(expanded_rows)


df_rainfall = fix_rainfall_states(df_rainfall)


df_rainfall = df_rainfall.drop_duplicates().reset_index(drop=True)

df_rainfall = df_rainfall[df_rainfall["State_Name"] != "Lakshadweep"]

print(df_rainfall["State_Name"].unique())


['Andaman and Nicobar Islands' 'Arunachal Pradesh' 'Assam' 'Meghalaya'
 'Naga Mani Mizo Tripura' 'Sub Himalayan West Bengal & Sikkim'
 'West Bengal' 'Odisha' 'Jharkhand' 'Bihar' 'Uttar Pradesh' 'Uttarakhand'
 'Haryana Delhi & Chandigarh' 'Punjab' 'Himachal Pradesh'
 'Jammu & Kashmir' 'Rajasthan' 'Madhya Pradesh' 'Gujarat'
 'Saurashtra & Kutch' 'Konkan & Goa' 'Maharashtra' 'Chhattisgarh'
 'Andhra Pradesh' 'Telangana' 'Tamil Nadu' 'Karnataka' 'Kerala']


In [ ]:
df_crops=pd.read_csv("crops.csv",encoding='latin1')
df_crops.head()
print(df_crops['State_Name'].unique())

['Andaman and Nicobar Islands' 'Andhra Pradesh' 'Arunachal Pradesh'
 'Assam' 'Bihar' 'Chandigarh' 'Chhattisgarh' 'Dadra and Nagar Haveli'
 'Goa' 'Gujarat' 'Haryana' 'Himachal Pradesh' 'Jammu and Kashmir '
 'Jharkhand' 'Karnataka' 'Kerala' 'Madhya Pradesh' 'Maharashtra' 'Manipur'
 'Meghalaya' 'Mizoram' 'Nagaland' 'Odisha' 'Puducherry' 'Punjab'
 'Rajasthan' 'Sikkim' 'Tamil Nadu' 'Telangana ' 'Tripura' 'Uttar Pradesh'
 'Uttarakhand' 'West Bengal']


In [ ]:
'''Making sure there are no discprepencies'''
df_crops['State_Name'] = df_crops['State_Name'].str.strip()
df_rainfall['State_Name'] = df_rainfall['State_Name'].str.strip()


In [ ]:
print(df_crops.columns)
print(df_rainfall.columns)
df_crops['State_Name'] = df_crops['State_Name'].str.strip()
df_rainfall['State_Name'] = df_rainfall['State_Name'].str.strip()
df_crops['Crop_Year'] = pd.to_numeric(df_crops['Crop_Year'], errors='coerce')
df_rainfall['Crop_Year'] = pd.to_numeric(df_rainfall['Crop_Year'], errors='coerce')

Index(['State_Name', 'District_Name', 'Crop_Year', 'Season', 'Crop', 'Area',
       'Production'],
      dtype='object')
Index(['State_Name', 'Crop_Year', 'Whole Year', 'Winter', 'Summer', 'Kharif',
       'Rabi'],
      dtype='object')


In [ ]:
'''FIRST MERGE : Rainfall + Crops Dataset'''
season_mapping = {
    'Kharif': 'Kharif',
    'Rabi': 'Rabi',
    'Winter': 'Winter',
    'Summer': 'Summer',
    'Whole Year': 'Whole Year'  
}
df_merged = df_crops.merge(df_rainfall, left_on=["State_Name", "Crop_Year"], right_on=["State_Name", "Crop_Year"], how="left")

df_merged["Rainfall"] = df_merged.apply(lambda row: row[season_mapping.get(row["Season"].strip(), "Whole Year")], axis=1)



In [ ]:
'''Before Second Merge, making sure there are no discrepencies'''
df_crops.columns = df_crops.columns.str.strip()
final_aqi_df.columns = final_aqi_df.columns.str.strip()


final_aqi_df["Crop_Year"] = final_aqi_df["Crop_Year"].astype(int)
df_crops["Crop_Year"] = df_crops["Crop_Year"].astype(int)
df_crops["State_Name"] = df_crops["State_Name"].str.strip()
final_aqi_df["State_Name"] = final_aqi_df["State_Name"].str.strip()

nan_count = df_crops['Crop_Year'].isna().sum()
print(f"Number of NaN values in crops : {nan_count}")
nan_count = df_merged['Crop_Year'].isna().sum()
print(f"Number of NaN values in merged: {nan_count}")

df_merged.head()


Number of NaN values in crops : 0
Number of NaN values in merged: 0


,State_Name,District_Name,Crop_Year,Season,Crop,Area,Production,Whole Year,Winter,Summer,Kharif,Rabi,Rainfall
0,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Arecanut,1254.0,2000.0,2763.2,227.2,812.2,1244.2,479.5,1244.2
1,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Other Kharif pulses,2.0,1.0,2763.2,227.2,812.2,1244.2,479.5,1244.2
2,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Rice,102.0,321.0,2763.2,227.2,812.2,1244.2,479.5,1244.2
3,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Banana,176.0,641.0,2763.2,227.2,812.2,1244.2,479.5,2763.2
4,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Cashewnut,720.0,165.0,2763.2,227.2,812.2,1244.2,479.5,2763.2


In [ ]:
'''FINAL MERGE: Rainfall + Crops + AQI'''
final_aqi_df.rename(columns={'season':'Season'},inplace=True)
df_merged['Season']=df_merged['Season'].str.strip()
df_new_merged = df_merged.merge(final_aqi_df, on=["State_Name", "Crop_Year", "Season"], how="left")
df_new_merged.head()

,State_Name,District_Name,Crop_Year,Season,Crop,Area,Production,Whole Year,Winter,Summer,Kharif,Rabi,Rainfall,so2,no2,rspm,spm,pm2_5
0,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Arecanut,1254.0,2000.0,2763.2,227.2,812.2,1244.2,479.5,1244.2,NaN,NaN,NaN,NaN,NaN
1,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Other Kharif pulses,2.0,1.0,2763.2,227.2,812.2,1244.2,479.5,1244.2,NaN,NaN,NaN,NaN,NaN
2,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Rice,102.0,321.0,2763.2,227.2,812.2,1244.2,479.5,1244.2,NaN,NaN,NaN,NaN,NaN
3,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Banana,176.0,641.0,2763.2,227.2,812.2,1244.2,479.5,2763.2,NaN,NaN,NaN,NaN,NaN
4,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Cashewnut,720.0,165.0,2763.2,227.2,812.2,1244.2,479.5,2763.2,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_new_merged.fillna(0, inplace=True)
df_new_merged.head()

,State_Name,District_Name,Crop_Year,Season,Crop,Area,Production,Whole Year,Winter,Summer,Kharif,Rabi,Rainfall,so2,no2,rspm,spm,pm2_5
0,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Arecanut,1254.0,2000.0,2763.2,227.2,812.2,1244.2,479.5,1244.2,0.0,0.0,0.0,0.0,0.0
1,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Other Kharif pulses,2.0,1.0,2763.2,227.2,812.2,1244.2,479.5,1244.2,0.0,0.0,0.0,0.0,0.0
2,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Rice,102.0,321.0,2763.2,227.2,812.2,1244.2,479.5,1244.2,0.0,0.0,0.0,0.0,0.0
3,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Banana,176.0,641.0,2763.2,227.2,812.2,1244.2,479.5,2763.2,0.0,0.0,0.0,0.0,0.0
4,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Cashewnut,720.0,165.0,2763.2,227.2,812.2,1244.2,479.5,2763.2,0.0,0.0,0.0,0.0,0.0


In [ ]:
'''Dataset Created'''
df_new_merged.to_csv("final_merged.csv", index=False)